<a href="https://colab.research.google.com/github/sdam-au/PHI_ETL/blob/master/scripts/1_1_py_PHI_EXTRACTION_FROM_ZIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import math
import pandas as pd

import sys
### we do a lot of requests during the scrapping. Some of them with requests package, some of them with urllib
import requests
from urllib.request import urlopen 
from urllib.parse import quote  
from bs4 import BeautifulSoup
import xml.etree.cElementTree as ET

import zipfile
import io

# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import getpass

import datetime as dt
# for simple paralel computing:
from concurrent.futures import ThreadPoolExecutor

In [0]:
!pip install sddk --upgrade
import sddk

In [3]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


# Extract the data from a zipfile on sciencedata.dk


In [0]:
### to get the zip file into a response
response = conf[0].get(conf[1] + "SDAM_data/PHI/PHI_Petra.zip")


In [0]:
### check that the response is functional
response

<Response [200]>

In [0]:
%%time
zipped = zipfile.ZipFile(io.BytesIO(response.content))

CPU times: user 29.1 ms, sys: 3.7 ms, total: 32.8 ms
Wall time: 37.7 ms


In [0]:
# how many files we have?
namelist = zipped.namelist()[1:] ### index "0" is for main directory
len(namelist)

3363

In [0]:
namelist = [name for name in namelist if not "._" in name]
namelist = [name for name in namelist if  ".csv" in name]
namelist = [name for name in namelist if  "output_parsed" in name]

len(namelist)

743

In [0]:
namelist[:100]

['PHI/output_parsed/Clinton--Sacred-Officials.csv',
 'PHI/output_parsed/Clara-Rhodos.csv',
 'PHI/output_parsed/ClRev.csv',
 'PHI/output_parsed/Christol-Drew-Bear--Castellum-rom..csv',
 'PHI/output_parsed/Chiron.csv',
 'PHI/output_parsed/Chios.csv',
 'PHI/output_parsed/Chaniotis--Vertr-ge.csv',
 'PHI/output_parsed/Cesnola--Cyprus.csv',
 'PHI/output_parsed/CahArch.csv',
 'PHI/output_parsed/Cabanes--L--pire.csv',
 'PHI/output_parsed/CSCA.csv',
 'PHI/output_parsed/CRAI.csv',
 'PHI/output_parsed/CPh.csv',
 'PHI/output_parsed/CIRB.csv',
 'PHI/output_parsed/CIL-III.csv',
 'PHI/output_parsed/CIJud.-II.csv',
 'PHI/output_parsed/CIG.csv',
 'PHI/output_parsed/CID.csv',
 'PHI/output_parsed/CEG.csv',
 'PHI/output_parsed/CCET.csv',
 'PHI/output_parsed/ByzantologSynedr-9-1953.csv',
 'PHI/output_parsed/Byzantion.csv',
 'PHI/output_parsed/Buresch--Aus-Lydien.csv',
 'PHI/output_parsed/Budde--Ant.Mosaiken-I.csv',
 'PHI/output_parsed/Brun-mid--Inschriften.csv',
 'PHI/output_parsed/Brixhe--Dial.gr.Pamph..c

# Test with one file

In [0]:
test = pd.read_csv(io.BytesIO(zipped.read("PHI/output_parsed/CEG.csv")))

In [0]:
test.head()

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,region,stoich,sinistr,subregion,date,other_info_0,other_info_1,other_info_2,other_info_3,other_info_4
0,/text/328151?location=1&patt=&bookid=303&offse...,CEG,I 109,Regions,CEG I 109,NaN,Funerary epigram for Dermys and Kitylos. Lime...,5,"i, left figure.1\n\nii, right figure.2\n\niii,...",Δέρμυς.\n\nΚιτύλος.\n\nἈμφάλκες ἔστασ’ ἐπὶ Κιτ...,Boiotia,NaN,NaN,Tanagra,ca. 600 BC,ca. 600 BC,IG VII 579,GVI 137,"LSAG² 92; 94, 8",NaN
1,/text/328152?location=1&patt=&bookid=303&offse...,CEG,I 110,Regions,CEG I 110,NaN,"Funerary epigram for Kallias, son of Aigisthos...",4,1\n\n\n,Καλλία\nΑἰγίθοιο {²⁶Αἰγίσθοιο}²⁶·\nτὺ δ’ εὖ πρ...,Boiotia,NaN,NaN,Haliartos (Kastri Maziou),ca. 500 BC?,ca. 500 BC?,IG VII 2852,GVI 62,NaN,NaN
2,/text/328153?location=1&patt=&bookid=303&offse...,CEG,I 111,Regions,CEG I 111,NaN,Funerary epigram for Gathon. Limestone tablet...,1,1,[π]αῖδες̣ ἐ̣[ποίεσ]α̣ν μ̣νᾶ̣μ̣’ ἐνθάδ̣[ε] πατρ...,Boiotia,NaN,NaN,Tanagra,ca. 500 BC?,ca. 500 BC?,IG VII 3501 (a),GVI 162,SEG 15.316,NaN
3,/text/328154?location=1&patt=&bookid=303&offse...,CEG,I 112,Regions,CEG I 112,NaN,Funerary epigram for Phanes. Funerary stele. ...,2,1\n,ἀσστοῖ[ς] καὶ χσένοισι Φάνες φίλος [ἐνθάδε κεῖ...,Boiotia,NaN,NaN,Thisbe,ca. 500 BC?,ca. 500 BC?,IG VII 2247,GVI 321 (cf. SEG 15.327),"CEG I, Add. p. 261",NaN
4,/text/328155?location=1&patt=&bookid=303&offse...,CEG,I 113,Regions,CEG I 113,NaN,"Funerary epigram for Oligeides, son of Osthilo...",6,1\n\n\n\n5\n,μνᾶμ’ ἐπ’ Ὀλιγέ-\nδαι {ι} μ’ ὁ πατὲρ ἐ-\nπέθεκ...,Boiotia,NaN,NaN,Thespiai,ca. 500-480 BC?,Palaiopanagia,ca. 500-480 BC?,IG VII 1880,GVI 152,SEG 15.325


# Extracting the whole dataset

In [0]:
dfs = []
for filename in namelist:
  df = pd.read_csv(io.BytesIO(zipped.read(filename)))
  dfs.append(df)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
dfs = pd.concat(dfs)
dfs.shape

(214995, 25)

In [0]:
### so let look at what is somewhere in the middle...
dfs.iloc[10000:10005]

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,region,stoich,sinistr,subregion,date,other_info_0,other_info_1,other_info_2,other_info_3,other_info_4,Unnamed: 0,petra,X260,17.12.2019 16:59,file:///home/petra/.config/libreoffice/4;
55,/text/256730?location=1040&patt=&bookid=482&bo...,Alabanda,49*5,Regions\nAsia Minor\nCaria,Alabanda 49*5,NaN,NaN,3.0,1\n\n,"Ἐπαφρᾶ #⁹³⁷ τραπεζείτου,\nπατρὸς δὲ Φιλουμενοῦ...",LW 557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,/text/256731?location=1040&patt=&bookid=482&of...,Alabanda,50,Regions\nAsia Minor\nCaria,Alabanda 50,NaN,Funerary inscription of Ephebos Neikanoros ...,2.0,1\n,Ἐφήβου τοῦ\nΝεικάνορος.,"BCH 1894, 34",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,/text/256732?location=1040&patt=&bookid=482&of...,Alabanda,51,Regions\nAsia Minor\nCaria,Alabanda 51,NaN,NaN,1.0,1,Εὐκάρπου οἰκείου Ἀριστογένου.,LW 563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,/text/256733?location=1040&patt=&bookid=482&of...,Alabanda,52,Regions\nAsia Minor\nCaria,Alabanda 52,NaN,Reclassified as Hyllarima 24. \n \n,1.0,0,{²reclassified as Hyllarima 24}²,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,/text/256734?location=1040&patt=&bookid=482&of...,Alabanda,53,Regions\nAsia Minor\nCaria,Alabanda 53,NaN,Funerary inscription of Eutyches [D]o[x]ipa...,2.0,1\n,Εὐτύχου [Δ]ο[ξ]ιπάτρου·\nvacat ζῇ.,LW 562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# what are the columns?
dfs.columns

Index(['URL', 'Book', 'Text', 'hdr1', 'hdr2', 'tildeinfo', 'note', 'lines',
       'metadata', 'data', 'region', 'stoich', 'sinistr', 'subregion', 'date',
       'other_info_0', 'other_info_1', 'other_info_2', 'other_info_3',
       'other_info_4', 'Unnamed: 0', 'petra', 'X260', '17.12.2019 16:59',
       'file:///home/petra/.config/libreoffice/4;'],
      dtype='object')

In [0]:
# ok, some of these columns are probably useless
# check what contain
dfs["file:///home/petra/.config/libreoffice/4;"].unique().tolist()

[nan]

In [0]:
dfs["X260"].unique()

array([nan], dtype=object)

In [0]:
dfs["17.12.2019 16:59"].unique()

array([nan], dtype=object)

In [0]:
dfs.reset_index(inplace=True)

In [0]:
# remove the strange columns
dfs.drop(['Unnamed: 0', 'petra', 'X260', '17.12.2019 16:59',
       'file:///home/petra/.config/libreoffice/4;', 'level_0', 'index'], axis=1, inplace=True)


In [0]:
dfs.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,region,stoich,sinistr,subregion,date,other_info_0,other_info_1,other_info_2,other_info_3,other_info_4
0,/text/237766?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials",10,Regions\nAttica (IG I-III),"Clinton, Sacred Officials 10",NaN,NaN,46.0,5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20\n\n\n\n\...,[․․․․․12․․․․․] ὀβολ[ὸν ․․5․․]\n[․․․․․12․․․․․]ο...,Att.,stoich. 23,NaN,Ath.: Eleusinion,ca. 460 BC,ca. 460 BC,"IG I² 6,C","I(3).6,C ll. 5-50",NaN,NaN
1,/text/237767?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","18,H5",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 18,H5",NaN,NaN,35.0,\n1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n...,corona\nθ[εοί].\nΕ̣ὐθ̣[․․6․․․]θ̣ωνος Ἐλευσίνιο...,Att.,stoich. 25,NaN,Eleusis,ca. mid-4th c. BC,ca. mid-4th c. BC,IG II² 1188,NaN,NaN,NaN
2,/text/237768?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","24,H11",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 24,H11",NaN,NaN,26.0,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20...,ἐπὶ Λυσιάδου ἄρχοντος Π̣[υανοψ]ιῶνος ἕκτ[ει ἐπ...,Att.,non-stoich.,NaN,Ath.: Agora,NaN,148/7? BC,"Hsp.11.1942.293,58 [SEG 19.124]",NaN,NaN,NaN
3,/text/237769?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","30,H19",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 30,H19",NaN,NaN,5.0,1\n\n\n\n5,[Τ. Φλ]ά̣β. Πάντα[ινον]\n[Γαργ]ήττιον vac.\n[․...,Att.,NaN,NaN,Athens,ca. 100 AD?,ca. 100 AD?,"Hsp.35.1966.247,8 +","AD 25.1970.187,6 [SEG 23.114]",SEG 29.180,NaN
4,/text/237770?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","31,H20",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 31,H20",NaN,NaN,14.0,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n,[κ]α̣θ’ ὑπομνη[ματι]-\nσμὸν τῆς ἐξ Ἀ[ρείου]\nπ...,Att.,NaN,NaN,Eleusis,166/7 AD,166/7 AD,IG II² 3984,NaN,NaN,NaN


In [0]:
sddk.write_file("SDAM_data/PHI/PHI_merged.json", dfs, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_merged.json"


In [0]:
# produce a sample:
sddk.write_file("SDAM_data/PHI/PHI_sample.json", dfs[:10000], conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_sample.json"


# Reading the files back and correcting encoding

In [0]:
PHI = sddk.read_file("SDAM_data/PHI/PHI_merged.json", "df", conf)

In [0]:
### there was an update in the sddk package, which now manages encoding on its own
sddk.write_file("SDAM_data/PHI/PHI_sample_utf8.json", PHI[:10000], conf)

A file with the same name ("PHI_sample_utf8.json") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_sample_utf8.json"


In [0]:
### and now do the same with the whole dataset
sddk.write_file("SDAM_data/PHI/PHI_merged_utf8.json", PHI, conf)

A file with the same name ("PHI_merged_utf8.json") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_merged_utf8.json"


# Exporting list of inscriptions


In [0]:
PHI = sddk.read_file("SDAM_data/PHI/PHI_merged.json", "df", conf)

In [5]:
inscriptions_list = [url.partition("text/")[2].partition("?")[0] for url in  PHI["URL"]]
print(inscriptions_list[:100])

['237766', '237767', '237768', '237769', '237770', '237771', '237772', '237773', '237774', '237775', '237776', '237777', '237778', '237779', '237780', '237781', '237782', '237783', '237784', '237785', '237786', '237787', '237788', '237789', '237790', '237791', '237792', '237793', '237794', '237795', '237796', '237797', '237798', '237799', '237800', '237801', '237802', '237803', '237804', '237805', '237806', '237807', '237808', '237809', '237810', '237811', '237812', '190454', '190455', '190457', '190458', '190459', '190460', '190461', '190462', '190463', '190464', '190465', '190466', '190467', '190468', '190469', '190470', '190471', '190472', '190473', '190474', '190475', '190476', '190477', '190478', '190479', '190480', '190481', '190482', '190483', '190484', '190485', '190486', '190487', '190488', '190489', '190490', '190491', '190492', '190493', '190494', '190495', '190496', '190497', '190498', '190499', '190500', '190501', '190502', '186278', '190503', '190504', '190505', '190506']

In [0]:
PHI["inscription ID"] = inscriptions_list

In [13]:
PHI.head()

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,region,stoich,sinistr,subregion,date,other_info_0,other_info_1,other_info_2,other_info_3,other_info_4,inscription ID
0,/text/237766?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials",10,Regions\nAttica (IG I-III),"Clinton, Sacred Officials 10",None,None,46.0,5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20\n\n\n\n\...,[․․․․․12․․․․․] ὀβολ[ὸν ․․5․․]\n[․․․․․12․․․․․]ο...,Att.,stoich. 23,None,Ath.: Eleusinion,ca. 460 BC,ca. 460 BC,"IG I² 6,C","I(3).6,C ll. 5-50",None,None,237766
1,/text/237767?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","18,H5",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 18,H5",None,None,35.0,\n1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n...,corona\nθ[εοί].\nΕ̣ὐθ̣[․․6․․․]θ̣ωνος Ἐλευσίνιο...,Att.,stoich. 25,None,Eleusis,ca. mid-4th c. BC,ca. mid-4th c. BC,IG II² 1188,None,None,None,237767
2,/text/237768?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","24,H11",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 24,H11",None,None,26.0,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20...,ἐπὶ Λυσιάδου ἄρχοντος Π̣[υανοψ]ιῶνος ἕκτ[ει ἐπ...,Att.,non-stoich.,None,Ath.: Agora,None,148/7? BC,"Hsp.11.1942.293,58 [SEG 19.124]",None,None,None,237768
3,/text/237769?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","30,H19",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 30,H19",None,None,5.0,1\n\n\n\n5,[Τ. Φλ]ά̣β. Πάντα[ινον]\n[Γαργ]ήττιον vac.\n[․...,Att.,None,None,Athens,ca. 100 AD?,ca. 100 AD?,"Hsp.35.1966.247,8 +","AD 25.1970.187,6 [SEG 23.114]",SEG 29.180,None,237769
4,/text/237770?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","31,H20",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 31,H20",None,None,14.0,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n,[κ]α̣θ’ ὑπομνη[ματι]-\nσμὸν τῆς ἐξ Ἀ[ρείου]\nπ...,Att.,None,None,Eleusis,166/7 AD,166/7 AD,IG II² 3984,None,None,None,237770


In [9]:
len(inscriptions_list)

214995

In [10]:
# number of unique values
len(list(set(inscriptions_list)))

214995

In [22]:
inscriptions_list_sorted = sorted([int(ins) for ins in inscriptions_list])
print(inscriptions_list_sorted[:1000])

[1754, 1755, 1756, 1757, 1758, 1759, 1760, 1761, 1762, 1763, 1764, 1765, 1766, 1767, 1768, 1769, 1770, 1771, 1772, 1773, 1774, 1775, 1776, 1777, 1778, 1779, 1780, 1781, 1782, 1783, 1784, 1785, 1786, 1787, 1788, 1789, 1790, 1791, 1792, 1793, 1794, 1795, 1796, 1797, 1798, 1799, 1800, 1801, 1802, 1803, 1804, 1805, 1806, 1807, 1808, 1809, 1810, 1811, 1812, 1813, 1814, 1815, 1816, 1817, 1818, 1819, 1820, 1821, 1822, 1823, 1824, 1825, 1826, 2219, 2220, 2221, 2222, 2223, 2224, 2225, 2226, 2227, 2228, 2229, 2230, 2231, 2232, 2233, 2234, 2235, 2236, 2237, 2238, 2239, 2240, 2241, 2242, 2243, 2244, 2245, 2246, 2247, 2248, 2249, 2250, 2251, 2252, 2253, 2254, 2255, 2256, 2257, 2258, 2259, 2260, 2261, 2262, 2263, 2264, 2265, 2266, 2267, 2268, 2269, 2270, 2271, 2272, 2273, 2274, 2275, 2276, 2277, 2278, 2279, 2280, 2281, 2282, 2283, 2284, 2285, 2286, 2287, 2288, 2289, 2290, 2291, 2292, 2293, 2294, 2295, 2296, 2297, 2298, 2299, 2300, 2301, 2302, 2303, 2304, 2305, 2306, 2307, 2308, 2309, 2310, 2311, 231

In [24]:
sddk.write_file("SDAM_data/PHI/inscriptions_list_sorted.json", inscriptions_list_sorted, conf)

Your <class 'list'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/inscriptions_list_sorted.json"


# Testing: reading sample digested by R by to Python

In [0]:
# reading the json processed in R back to Python:
PHI_sample = sddk.read_file("SDAM_data/PHI/PHI_tibble.json", "df", conf)
PHI_sample.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,region,stoich,sinistr,subregion,date,other_info_0,other_info_1,other_info_2,other_info_3,other_info_4
0,[/text/237766?location=7&patt=&bookid=409&book...,"[Clinton, Sacred Officials]",[10],[Regions\nAttica (IG I-III)],"[Clinton, Sacred Officials 10]",{},{},[46],[5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20\n\n\n\n...,[[․․․․․12․․․․․] ὀβολ[ὸν ․․5․․]\n[․․․․․12․․․․․]...,[Att.],[stoich. 23],{},[Ath.: Eleusinion],[ca. 460 BC],[ca. 460 BC],"[IG I² 6,C]","[I(3).6,C ll. 5-50]",{},{}
1,[/text/237767?location=7&patt=&bookid=409&book...,"[Clinton, Sacred Officials]","[18,H5]",[Regions\nAttica (IG I-III)],"[Clinton, Sacred Officials 18,H5]",{},{},[35],[\n1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\...,[corona\nθ[εοί].\nΕ̣ὐθ̣[․․6․․․]θ̣ωνος Ἐλευσίνι...,[Att.],[stoich. 25],{},[Eleusis],[ca. mid-4th c. BC],[ca. mid-4th c. BC],[IG II² 1188],{},{},{}
2,[/text/237768?location=7&patt=&bookid=409&book...,"[Clinton, Sacred Officials]","[24,H11]",[Regions\nAttica (IG I-III)],"[Clinton, Sacred Officials 24,H11]",{},{},[26],[1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n2...,[ἐπὶ Λυσιάδου ἄρχοντος Π̣[υανοψ]ιῶνος ἕκτ[ει ἐ...,[Att.],[non-stoich.],{},[Ath.: Agora],{},[148/7? BC],"[Hsp.11.1942.293,58 [SEG 19.124]]",{},{},{}
3,[/text/237769?location=7&patt=&bookid=409&book...,"[Clinton, Sacred Officials]","[30,H19]",[Regions\nAttica (IG I-III)],"[Clinton, Sacred Officials 30,H19]",{},{},[5],[1\n\n\n\n5],[[Τ. Φλ]ά̣β. Πάντα[ινον]\n[Γαργ]ήττιον vac.\n[...,[Att.],{},{},[Athens],[ca. 100 AD?],[ca. 100 AD?],"[Hsp.35.1966.247,8 +]","[AD 25.1970.187,6 [SEG 23.114]]",[SEG 29.180],{}
4,[/text/237770?location=7&patt=&bookid=409&book...,"[Clinton, Sacred Officials]","[31,H20]",[Regions\nAttica (IG I-III)],"[Clinton, Sacred Officials 31,H20]",{},{},[14],[1\n\n\n\n5\n\n\n\n\n10\n\n\n\n],[[κ]α̣θ’ ὑπομνη[ματι]-\nσμὸν τῆς ἐξ Ἀ[ρείου]\n...,[Att.],{},{},[Eleusis],[166/7 AD],[166/7 AD],[IG II² 3984],{},{},{}
